# Training a model for catchment classification. 
* kategoryzacja zlewni oparta o większość dostępnych ceech zlewni z plików inp i rpt

# Imports

In [1]:
import swmmio
import pyswmm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

desired_width = 500
pd.set_option("display.width", desired_width)
np.set_printoptions(linewidth=desired_width)
pd.set_option("display.max_columns", 30)

In [2]:
classes = pd.DataFrame(
    data={
        "classes": [
            "compact_urban_development",
            "urban",
            "loose_urban_development",
            "wooded_area",
            "grassy",
            "loose_soil",
            "steep_area",
        ]
    }
)
classes

,classes
0,compact_urban_development
1,urban
2,loose_urban_development
3,wooded_area
4,grassy
5,loose_soil
6,steep_area


# Get files

In [3]:
INP_FILE = "dataset/subcatchment_dataset.inp"
RPT_FILE = "dataset/subcatchment_dataset.inp"

# Run simulation

In [4]:
with pyswmm.Simulation(INP_FILE) as sim:
    for step in sim:
        pass

# Read inp and rpt file as swmmio model object

In [5]:
model = swmmio.Model(INP_FILE)

## Get subcatchments data from the model

In [6]:
raw_subcatchments = model.subcatchments.dataframe
subcatchments = raw_subcatchments.copy()
subcatchments[:5]

,Raingage,Outlet,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,RouteTo,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff,coords
Name,,,,,,,,,,,,,,,,,,,,,,,,
S1,Raingage2,O1,0.50,20.00,300.00,5.00,0,0.150,0.41,1.27,5.08,70,OUTLET,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407,"[(777180.0, 592590.0), (777180.0, 592585.0), (777175.0, 592585.0), (777175.0, 592590.0), (777180.0, 592590.0)]"
S10,Raingage2,O1,1.87,45.00,136.75,15.00,0,0.013,0.15,1.27,5.08,90,OUTLET,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587,NaN
S100,Raingage2,O1,1.71,45.00,130.77,10.00,0,0.013,0.15,1.27,5.08,90,OUTLET,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581,NaN
S1000,Raingage2,O1,0.68,10.00,82.46,5.09,0,0.150,0.41,1.27,5.08,10,OUTLET,10.10,0.00,0.00,7.73,0.90,1.36,2.26,0.02,0.01,0.224,NaN
S1001,Raingage2,O1,0.26,83.33,50.99,45.00,0,0.013,0.05,1.27,5.08,80,OUTLET,10.10,0.00,0.00,1.04,8.23,0.65,8.88,0.02,0.02,0.879,NaN


### Drop unused columns

In [7]:
subcatchments.drop(['coords', 'RouteTo'], axis=1, inplace=True)
subcatchments

,Raingage,Outlet,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff
Name,,,,,,,,,,,,,,,,,,,,,,
S1,Raingage2,O1,0.50,20.00,300.00,5.00,0,0.150,0.41,1.27,5.08,70,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407
S10,Raingage2,O1,1.87,45.00,136.75,15.00,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587
S100,Raingage2,O1,1.71,45.00,130.77,10.00,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581
S1000,Raingage2,O1,0.68,10.00,82.46,5.09,0,0.150,0.41,1.27,5.08,10,10.10,0.00,0.00,7.73,0.90,1.36,2.26,0.02,0.01,0.224
S1001,Raingage2,O1,0.26,83.33,50.99,45.00,0,0.013,0.05,1.27,5.08,80,10.10,0.00,0.00,1.04,8.23,0.65,8.88,0.02,0.02,0.879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S995,Raingage2,O1,1.10,10.00,104.88,15.00,0,0.150,0.41,1.27,5.08,10,10.10,0.00,0.00,7.53,0.90,1.56,2.46,0.03,0.02,0.243
S996,Raingage2,O1,1.89,5.09,137.48,15.00,0,0.060,0.17,1.27,5.08,10,10.10,0.00,0.00,7.55,0.46,2.04,2.49,0.05,0.03,0.247
S997,Raingage2,O1,1.19,10.00,109.09,5.09,0,0.150,0.41,1.27,5.08,10,10.10,0.00,0.00,7.91,0.90,1.18,2.08,0.02,0.01,0.206


## Get categories

In [8]:
categories = model.inp.tags
categories

,Name,Tag
ElementType,,
Subcatch,S1,#comment
Subcatch,S2,compact_urban_development
Subcatch,S3,loose_soil
Subcatch,S4,wooded_area
Subcatch,S5,loose_urban_development
...,...,...
Subcatch,S997,grassy
Subcatch,S998,steep_area
Subcatch,S999,urban


### Add categories column to subcatchments DataFrame

In [9]:
# Reset the index of subcatchments DataFrame
subcatchments.reset_index(inplace=True)

# Merge the two DataFrames on the "Name" column
merged_df = subcatchments.merge(model.inp.tags, left_on="Name", right_on="Name", how="left")

# Rename the 'Tag' column to 'categories'
merged_df.rename(columns={"Tag": "categories"}, inplace=True)

# Set the index back to "Name"
merged_df.set_index("Name", inplace=True)

In [10]:
merged_df[:3]

,Raingage,Outlet,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff,categories
Name,,,,,,,,,,,,,,,,,,,,,,,
S1,Raingage2,O1,0.50,20.0,300.00,5.0,0,0.150,0.41,1.27,5.08,70,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407,#comment
S10,Raingage2,O1,1.87,45.0,136.75,15.0,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587,compact_urban_development
S100,Raingage2,O1,1.71,45.0,130.77,10.0,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581,compact_urban_development


In [11]:
merged_df.head()

,Raingage,Outlet,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff,categories
Name,,,,,,,,,,,,,,,,,,,,,,,
S1,Raingage2,O1,0.50,20.00,300.00,5.00,0,0.150,0.41,1.27,5.08,70,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407,#comment
S10,Raingage2,O1,1.87,45.00,136.75,15.00,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587,compact_urban_development
S100,Raingage2,O1,1.71,45.00,130.77,10.00,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581,compact_urban_development
S1000,Raingage2,O1,0.68,10.00,82.46,5.09,0,0.150,0.41,1.27,5.08,10,10.10,0.00,0.00,7.73,0.90,1.36,2.26,0.02,0.01,0.224,grassy
S1001,Raingage2,O1,0.26,83.33,50.99,45.00,0,0.013,0.05,1.27,5.08,80,10.10,0.00,0.00,1.04,8.23,0.65,8.88,0.02,0.02,0.879,steep_area


### Split data into features and target

In [12]:
X = merged_df.drop('categories', axis=1)
y = merged_df['categories']
print(X.shape, y.shape)

(1001, 22) (1001,)


In [13]:
X["TotalPrecip"] = pd.to_numeric(X["TotalPrecip"])
X["TotalRunon"] = pd.to_numeric(X["TotalRunon"])
X["TotalEvap"] = pd.to_numeric(X["TotalEvap"])
X["TotalInfil"] = pd.to_numeric(X["TotalInfil"])
X["PervRunoff"] = pd.to_numeric(X["PervRunoff"])

In [14]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1001 entries, S1 to S999
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Raingage       1001 non-null   object 
 1   Outlet         1001 non-null   object 
 2   Area           1001 non-null   float64
 3   PercImperv     1001 non-null   float64
 4   Width          1001 non-null   float64
 5   PercSlope      1001 non-null   float64
 6   CurbLength     1001 non-null   int64  
 7   N-Imperv       1001 non-null   float64
 8   N-Perv         1001 non-null   float64
 9   S-Imperv       1001 non-null   float64
 10  S-Perv         1001 non-null   float64
 11  PctZero        1001 non-null   int64  
 12  TotalPrecip    1001 non-null   float64
 13  TotalRunon     1001 non-null   float64
 14  TotalEvap      1001 non-null   float64
 15  TotalInfil     1001 non-null   float64
 16  ImpervRunoff   1001 non-null   float64
 17  PervRunoff     1001 non-null   float64
 18  TotalRunoffI

In [15]:
X = pd.get_dummies(X, drop_first=True)
X.head()

,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff
Name,,,,,,,,,,,,,,,,,,,,
S1,0.50,20.00,300.00,5.00,0,0.150,0.41,1.27,5.08,70,10.1,0.0,0.0,5.91,1.95,2.17,4.12,0.02,0.02,0.407
S10,1.87,45.00,136.75,15.00,0,0.013,0.15,1.27,5.08,90,10.1,0.0,0.0,4.13,4.50,1.43,5.93,0.11,0.09,0.587
S100,1.71,45.00,130.77,10.00,0,0.013,0.15,1.27,5.08,90,10.1,0.0,0.0,4.18,4.50,1.38,5.87,0.10,0.08,0.581
S1000,0.68,10.00,82.46,5.09,0,0.150,0.41,1.27,5.08,10,10.1,0.0,0.0,7.73,0.90,1.36,2.26,0.02,0.01,0.224
S1001,0.26,83.33,50.99,45.00,0,0.013,0.05,1.27,5.08,80,10.1,0.0,0.0,1.04,8.23,0.65,8.88,0.02,0.02,0.879


In [16]:
# Create a MinMaxScaler instance
scaler = MinMaxScaler()

# Fit the scaler to the data and transform the data
X_normalized = scaler.fit_transform(X)

# Convert the normalized data back to a dataframe with the same column names
X_normalized_df = pd.DataFrame(X_normalized, columns=X.columns, index=X.index)

In [17]:
X_normalized_df.head()

,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff
Name,,,,,,,,,,,,,,,,,,,,
S1,0.246231,0.218148,1.000000,0.044995,0.0,0.354005,0.480000,0.0,0.0,0.764706,0.0,0.0,0.0,0.608479,0.216687,0.683871,0.392630,0.111111,0.142857,0.391528
S10,0.934673,0.526790,0.437069,0.213515,0.0,0.000000,0.133333,0.0,0.0,1.000000,0.0,0.0,0.0,0.386534,0.534247,0.445161,0.622618,0.611111,0.642857,0.622593
S100,0.854271,0.526790,0.416448,0.129255,0.0,0.000000,0.133333,0.0,0.0,1.000000,0.0,0.0,0.0,0.392768,0.534247,0.429032,0.614994,0.555556,0.571429,0.614891
S1000,0.336683,0.094691,0.249862,0.046512,0.0,0.354005,0.480000,0.0,0.0,0.058824,0.0,0.0,0.0,0.835411,0.085928,0.422581,0.156290,0.111111,0.071429,0.156611
S1001,0.125628,1.000000,0.141345,0.719077,0.0,0.000000,0.000000,0.0,0.0,0.882353,0.0,0.0,0.0,0.001247,0.998755,0.193548,0.997459,0.111111,0.142857,0.997433


### Encode target labels

In [18]:
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))

In [19]:
y_encoded

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

# Split the data into training and testing sets

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized_df, y_encoded, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (800, 20)
y_train shape: (800, 8)
X_test shape: (201, 20)
y_test shape: (201, 8)


# Build the model

In [21]:
model = Sequential()
model.add(Dense(16, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=y_train.shape[1], activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                336       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
Total params: 544
Trainable params: 544
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    x=X_train,
    y=y_train,
    epochs=100,
    validation_data=(X_test, y_test),
    validation_split=0.2,
    verbose=1,
    batch_size=16,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10)]
)

Epoch 1/100
50/50 [==============================] - 1s 8ms/step - loss: 2.1168 - accuracy: 0.0938 - val_loss: 1.9967 - val_accuracy: 0.2687
Epoch 2/100
50/50 [==============================] - 0s 3ms/step - loss: 1.9304 - accuracy: 0.3750 - val_loss: 1.8327 - val_accuracy: 0.6169
Epoch 3/100
50/50 [==============================] - 0s 3ms/step - loss: 1.7473 - accuracy: 0.5200 - val_loss: 1.6471 - val_accuracy: 0.7363
Epoch 4/100
50/50 [==============================] - 0s 3ms/step - loss: 1.5738 - accuracy: 0.5775 - val_loss: 1.4415 - val_accuracy: 0.7861
Epoch 5/100
50/50 [==============================] - 0s 3ms/step - loss: 1.3564 - accuracy: 0.6288 - val_loss: 1.2451 - val_accuracy: 0.7861
Epoch 6/100
50/50 [==============================] - 0s 2ms/step - loss: 1.1809 - accuracy: 0.6675 - val_loss: 1.0752 - val_accuracy: 0.7811
Epoch 7/100
50/50 [==============================] - 0s 3ms/step - loss: 1.1104 - accuracy: 0.6550 - val_loss: 0.9644 - val_accuracy: 0.8358
Epoch 8/100
5

In [23]:
# Evaluate ANN model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test set accuracy: {accuracy:.2f}")

7/7 [==============================] - 0s 2ms/step - loss: 0.0253 - accuracy: 1.0000
Test set accuracy: 1.00


In [24]:
# model.predict(X_test)
y_pred = model.predict(X_test)

7/7 [==============================] - 0s 1ms/step


In [25]:
metrics = pd.DataFrame(history.history)
metrics['epoch'] = history.epoch
metrics

,loss,accuracy,val_loss,val_accuracy,epoch
0,2.116811,0.09375,1.996739,0.268657,0
1,1.930363,0.37500,1.832740,0.616915,1
2,1.747340,0.52000,1.647108,0.736318,2
3,1.573808,0.57750,1.441517,0.786070,3
4,1.356449,0.62875,1.245062,0.786070,4
...,...,...,...,...,...
95,0.172196,0.94625,0.026607,1.000000,95
96,0.156622,0.95125,0.025964,1.000000,96
97,0.190026,0.93375,0.024610,1.000000,97
98,0.188896,0.93625,0.024194,1.000000,98


# Model evaluate

In [26]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['accuracy'], name='accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['loss'], name='loss'), row=1, col=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_accuracy'], name='val_accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_loss'], name='val_loss'), row=1, col=2)

fig.update_xaxes(title_text='epochs')
fig.update_yaxes(title_text='accuracy')
fig.update_layout(width=1000, title='Accuracy and Loss')
fig.show()

In [27]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(test_acc)

1.0


Predykcja na podstawie modelu.
* model.evaluate(y_true, y_pred) - pozwala obliczyć metryki modelu
* model.predict_classes() - pozwala zwrócić odpowiednio przewidziane klasy
* model.predict_proba(), model.predict() - pozwala zwrócić prawdopodobieństwo danej klasy

In [28]:
predictions = model.predict(X_test)
predictions

7/7 [==============================] - 0s 1ms/step


array([[8.4829134e-07, 1.3701143e-10, 9.9967754e-01, ..., 2.1793477e-08, 2.1865798e-08, 1.4135103e-05],
       [4.9498020e-05, 4.4177621e-05, 1.9142242e-05, ..., 1.5750881e-08, 4.6261344e-03, 8.2409731e-04],
       [3.5408000e-06, 1.8395278e-09, 9.9937952e-01, ..., 1.8111106e-07, 1.6817229e-07, 4.5402019e-05],
       ...,
       [3.8690558e-18, 1.3739363e-16, 3.5227493e-10, ..., 5.4100859e-11, 2.8511288e-21, 1.0000000e+00],
       [4.6557207e-06, 5.9808709e-09, 9.9938071e-01, ..., 7.8732995e-07, 2.2938669e-07, 1.4512446e-04],
       [2.3214897e-04, 1.3841258e-04, 1.2696438e-04, ..., 1.5424091e-07, 1.5679423e-02, 7.4573950e-04]], dtype=float32)

In [29]:
predictions_df = pd.DataFrame(predictions)
predictions_df

,0,1,2,3,4,5,6,7
0,8.482913e-07,1.370114e-10,9.996775e-01,3.183601e-06,3.041843e-04,2.179348e-08,2.186580e-08,0.000014
1,4.949802e-05,4.417762e-05,1.914224e-05,3.331641e-07,9.944366e-01,1.575088e-08,4.626134e-03,0.000824
2,3.540800e-06,1.839528e-09,9.993795e-01,1.419183e-05,5.570320e-04,1.811111e-07,1.681723e-07,0.000045
3,2.937949e-06,1.107188e-09,9.994031e-01,1.324157e-05,5.497016e-04,1.063220e-07,1.334730e-07,0.000031
4,5.710872e-06,7.486377e-09,9.992864e-01,1.577394e-05,5.493517e-04,8.326400e-07,3.003169e-07,0.000142
...,...,...,...,...,...,...,...,...
196,7.076921e-18,2.304002e-16,4.775598e-10,1.059084e-28,4.537669e-14,5.736029e-11,6.319476e-21,1.000000
197,7.576364e-06,1.369913e-08,9.991828e-01,2.009014e-05,5.864103e-04,1.468439e-06,4.521994e-07,0.000201
198,3.869056e-18,1.373936e-16,3.522749e-10,4.850987e-29,1.972961e-14,5.410086e-11,2.851129e-21,1.000000
199,4.655721e-06,5.980871e-09,9.993807e-01,1.175860e-05,4.566781e-04,7.873300e-07,2.293867e-07,0.000145


In [30]:
predictions_cls = predictions.argmax(axis=-1)
predictions_cls

array([2, 4, 2, 2, 2, 1, 2, 7, 3, 2, 3, 7, 2, 5, 5, 1, 5, 4, 2, 3, 7, 3, 7, 4, 7, 1, 2, 4, 6, 1, 2, 6, 6, 4, 6, 1, 7, 2, 7, 4, 2, 5, 7, 2, 2, 2, 7, 2, 3, 4, 1, 1, 4, 1, 3, 2, 7, 2, 7, 3, 2, 7, 2, 6, 7, 2, 3, 1, 3, 2, 2, 2, 4, 2, 2, 2, 1, 2, 2, 1, 7, 2, 2, 2, 6, 2, 5, 7, 2, 7, 6, 2, 3, 4, 2, 2, 2, 2, 6, 6, 3, 2, 6, 4, 5, 3, 6, 3, 2, 2, 1, 2, 2, 2, 6, 7, 3, 1, 2, 2, 6, 7, 2, 2, 6, 3, 2, 2, 3, 5, 4, 1, 6, 2, 1, 4, 7, 3, 3, 7, 6, 2, 7, 2, 2, 2, 4, 2, 4, 2, 3, 5, 2, 6, 1, 2, 5, 2, 4, 2, 4, 6, 2, 3,
       6, 5, 2, 1, 4, 2, 6, 2, 1, 5, 1, 2, 4, 2, 3, 2, 7, 1, 2, 3, 5, 6, 6, 1, 4, 2, 2, 2, 7, 7, 2, 1, 7, 2, 7, 2, 4], dtype=int64)

# Zapisanie modelu

In [34]:
model.save(r'C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\data\all_columns')

INFO:tensorflow:Assets written to: C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\data\all_columns\assets


# Załadowanie modelu

In [36]:
from tensorflow import keras
model = keras.models.load_model(r'C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\data\all_columns')

In [37]:
pred = model.predict(X_test)
pred

7/7 [==============================] - 0s 1ms/step


array([[8.4829134e-07, 1.3701143e-10, 9.9967754e-01, ..., 2.1793477e-08, 2.1865798e-08, 1.4135103e-05],
       [4.9498020e-05, 4.4177621e-05, 1.9142242e-05, ..., 1.5750881e-08, 4.6261344e-03, 8.2409731e-04],
       [3.5408000e-06, 1.8395278e-09, 9.9937952e-01, ..., 1.8111106e-07, 1.6817229e-07, 4.5402019e-05],
       ...,
       [3.8690558e-18, 1.3739363e-16, 3.5227493e-10, ..., 5.4100859e-11, 2.8511288e-21, 1.0000000e+00],
       [4.6557207e-06, 5.9808709e-09, 9.9938071e-01, ..., 7.8732995e-07, 2.2938669e-07, 1.4512446e-04],
       [2.3214897e-04, 1.3841258e-04, 1.2696438e-04, ..., 1.5424091e-07, 1.5679423e-02, 7.4573950e-04]], dtype=float32)